In [63]:
import pandas as pd
import numpy as np

In [64]:
def kd_adjust(data):        
    station_KD = []
    station_SD = []
    for measurement in data['Measurement']:
        if 'KD' in measurement:
            kd_idx = measurement
            station_KD = data['Station'][data['Measurement']==kd_idx]
        if 'SD' in measurement:
            sd_idx = measurement
            station_SD = data['Station'][data['Measurement']==sd_idx]
    for i in range(len(list(station_KD))):
        if list(station_KD)[i] == list(station_SD)[i]:
            data = data.drop(station_SD.index[i], axis = 0)
    data.index = range(len(data))
    return data

In [65]:
## import excel files set define df
ts_data = pd.read_excel("//dkcph1-stor2.dhi.dk/projects/11823492/03_MarineExplorer/Marine Explorer/MarineExplorerEco/Tools/Matlab/ModelPerformance/Performance_Plots/Ringkoebing Fjord/2020/23-Mar-2020_ModelPerformance_TS_2002_2016_misa_Ringkoebing Fjord.xlsx")
kd_data = pd.read_excel("//dkcph1-stor2.dhi.dk/projects/11823492/03_MarineExplorer/Marine Explorer/MarineExplorerEco/Tools/Matlab/ModelPerformance/Performance_Plots/Ringkoebing Fjord/2020/20-Mar-2020_ModelPerformance_KD_TS_2002_2016_misa_Ringkoebing Fjord.xlsx")

# define data frame
df = pd.DataFrame(ts_data[ts_data.columns[0:8]])
df = df.append(kd_data[kd_data.columns[0:8]], ignore_index=True)
df['Percent_Bias']=np.absolute(df['Percent_Bias'])

index_DO = []
index_TN = []
index_TP = []
for i in range(len(df)):
    if 'DO' in df['Measurement'][i]:
        index_DO.append(i)
    if 'TN' in df['Measurement'][i]:
        index_TN.append(i)
    if 'TP' in df['Measurement'][i]:
        index_TP.append(i)

stations_incl = df["Station"][index_TN].unique()
df_new = pd.DataFrame()
for i in range(len(stations_incl)):
    tmp1 = df.loc[df['Station']==stations_incl[i]]
    df_new = df_new.append(tmp1, ignore_index = True)

df_new['Standard']=0
df_new['Summer']=0
df_new['Winter']=0
for i in range(len(df_new)):
    if 'Standard' in df_new['Measurement'][i]:
        df_new['Standard'][i]=1 
    if 'Summer' in df_new['Measurement'][i]:
        df_new['Summer'][i]=1 
    if 'Winter' in df_new['Measurement'][i]:
        df_new['Winter'][i]=1 

df_year = df_new.loc[df_new['Standard']>0]
df_summer = df_new.loc[df_new['Summer']>0]
df_winter = df_new.loc[df_new['Winter']>0]

df_year_adj = kd_adjust(df_year)
df_summer_adj = kd_adjust(df_summer)

df_sumwint = pd.DataFrame()
df_sumwint = df_sumwint.append(df_summer_adj, ignore_index = True)
df_sumwint = df_sumwint.append(df_winter, ignore_index = True)
df_sumwint.append(df.loc[index_TN], ignore_index = True)
df_sumwint.append(df.loc[index_TP], ignore_index = True)
df_sumwint.append(df.loc[index_DO], ignore_index = True)

df_all = df_year_adj
df_all = df_all.append(df_summer_adj, ignore_index = True)
df_all = df_all.append(df_winter, ignore_index = True)




C:\Users\SONI\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\SONI\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\SONI\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\SONI\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\fram

In [66]:
def Pbias_perform(dataframe):
    perform_Pbias = []
    for i in range(len(dataframe)):
        if np.absolute(dataframe['Percent_Bias'][i]) <10:
            perform_Pbias.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['Percent_Bias'][i],dataframe['Total_Number_Of_Observations_1'][i],'Excellent', 'P_Bias'])
        if np.absolute(dataframe['Percent_Bias'][i]) >=10 and np.absolute(dataframe['Percent_Bias'][i]) <=20:
            perform_Pbias.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['Percent_Bias'][i],dataframe['Total_Number_Of_Observations_1'][i],'Very good', 'P_Bias'])
        if np.absolute(dataframe['Percent_Bias'][i]) >20 and np.absolute(dataframe['Percent_Bias'][i]) <=40:
            perform_Pbias.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['Percent_Bias'][i],dataframe['Total_Number_Of_Observations_1'][i],'Good', 'P_Bias'])    
        if np.absolute(dataframe['Percent_Bias'][i]) >40:
            perform_Pbias.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['Percent_Bias'][i],dataframe['Total_Number_Of_Observations_1'][i],'Poor', 'P_Bias'])  
    PerformPbias = pd.DataFrame(perform_Pbias)
    PerformPbias.columns = ['Station','Measurement','Stat_Value','Total_No_Obs','Performance', 'QMeasure']
    return PerformPbias
    
def SRC_perform(dataframe):
    perform_SRC = []
    for i in range(len(dataframe)):
        if dataframe['Spearman_Rank_Correlation'][i] >0.9:
            perform_SRC.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['Spearman_Rank_Correlation'][i],dataframe['Total_Number_Of_Observations_1'][i],'Excellent','SRC'])
        if dataframe['Spearman_Rank_Correlation'][i] <=0.9 and dataframe['Spearman_Rank_Correlation'][i] >=0.6:
            perform_SRC.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['Spearman_Rank_Correlation'][i],dataframe['Total_Number_Of_Observations_1'][i],'Very good','SRC'])
        if dataframe['Spearman_Rank_Correlation'][i] <0.6 and dataframe['Spearman_Rank_Correlation'][i] >=0.3:
            perform_SRC.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['Spearman_Rank_Correlation'][i],dataframe['Total_Number_Of_Observations_1'][i],'Good','SRC'])    
        if dataframe['Spearman_Rank_Correlation'][i] <0.3:
            perform_SRC.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['Spearman_Rank_Correlation'][i],dataframe['Total_Number_Of_Observations_1'][i],'Poor','SRC'])                                                                                                                                          
    PerformSRC = pd.DataFrame(perform_SRC)
    PerformSRC.columns = ['Station','Measurement','Stat_Value','Total_No_Obs','Performance', 'QMeasure']
    return PerformSRC
    
def MEF_perform(dataframe):       
    perform_MEF = []
    for i in range(len(dataframe)):
        if dataframe['Efficiency_Factor'][i] >0.8:
            perform_MEF.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['Efficiency_Factor'][i],dataframe['Total_Number_Of_Observations_1'][i],'Excellent','MEF'])
        if dataframe['Efficiency_Factor'][i] <=0.8 and dataframe['Efficiency_Factor'][i] >=0.5:
            perform_MEF.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['Efficiency_Factor'][i],dataframe['Total_Number_Of_Observations_1'][i],'Very good','MEF'])
        if dataframe['Efficiency_Factor'][i] <0.5 and dataframe['Efficiency_Factor'][i] >=0.2:
            perform_MEF.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['Efficiency_Factor'][i],dataframe['Total_Number_Of_Observations_1'][i],'Good','MEF'])    
        if dataframe['Efficiency_Factor'][i] <0.2:
            perform_MEF.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['Efficiency_Factor'][i],dataframe['Total_Number_Of_Observations_1'][i],'Poor','MEF'])  
    PerformMEF = pd.DataFrame(perform_MEF)
    PerformMEF.columns = ['Station','Measurement','Stat_Value','Total_No_Obs','Performance', 'QMeasure']
    return PerformMEF
    

def CF_perform(dataframe):
    perform_CF = []
    for i in range(len(dataframe)):
        if dataframe['Cost_Function'][i] <=0.4:
            perform_CF.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['Cost_Function'][i],dataframe['Total_Number_Of_Observations_1'][i],'Excellent','CF'])
        if dataframe['Cost_Function'][i] >0.4 and dataframe['Cost_Function'][i] <=1:
            perform_CF.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['Cost_Function'][i],dataframe['Total_Number_Of_Observations_1'][i],'Very good','CF'])
        if dataframe['Cost_Function'][i] >1 and dataframe['Cost_Function'][i] <3:
            perform_CF.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['Cost_Function'][i],dataframe['Total_Number_Of_Observations_1'][i],'Good','CF'])    
        if dataframe['Cost_Function'][i] >=3:
            perform_CF.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['Cost_Function'][i],dataframe['Total_Number_Of_Observations_1'][i],'Poor','CF'])  
    PerformCF = pd.DataFrame(perform_CF)
    PerformCF.columns = ['Station','Measurement','Stat_Value','Total_No_Obs','Performance', 'QMeasure']
    return PerformCF
    
def R2_perform(dataframe):
    perform_R2 = []
    for i in range(len(dataframe)):
        if dataframe['R2'][i] >0.65:
            perform_R2.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['R2'][i],dataframe['Total_Number_Of_Observations_1'][i],'Excellent','R2'])
        if dataframe['R2'][i] <=0.65 and dataframe['R2'][i] >=0.35:
            perform_R2.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['R2'][i],dataframe['Total_Number_Of_Observations_1'][i],'Very good','R2'])
        if dataframe['R2'][i] <0.35 and dataframe['R2'][i] >=0.175:
            perform_R2.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['R2'][i],dataframe['Total_Number_Of_Observations_1'][i],'Good','R2'])    
        if dataframe['R2'][i] <0.175:
            perform_R2.append([dataframe['Station'][i],dataframe['Measurement'][i],dataframe['R2'][i],dataframe['Total_Number_Of_Observations_1'][i],'Poor','R2'])  
    PerformR2 = pd.DataFrame(perform_R2)
    PerformR2.columns = ['Station','Measurement','Stat_Value','Total_No_Obs','Performance', 'QMeasure']
    return PerformR2

In [67]:
def collect_param_df(dataframe, param):
    param_perform = pd.DataFrame()
    for i in range(len(dataframe)):
        if param in dataframe['Measurement'][i]:
            tmp = pd.DataFrame(dataframe.loc[i]).T
            param_perform = param_perform.append(tmp, ignore_index = True)
    
    return param_perform

def distribution(data_stat, Qmeasure):
    A = (len(data_stat[data_stat['Performance']=='Excellent'])/len(data_stat))*100
    B = (len(data_stat[data_stat['Performance']=='Very good'])/len(data_stat))*100
    C = (len(data_stat[data_stat['Performance']=='Good'])/len(data_stat))*100
    D = (len(data_stat[data_stat['Performance']=='Poor'])/len(data_stat))*100
    AbovePoor = (A+B+C)
    Dist_stat = pd.DataFrame([round(A,1), round(B,1), round(C,1), round(D,1), round(AbovePoor,2), Qmeasure]).T
    Dist_stat.columns = ['Excellent','Very good', 'Good','Poor', 'Above poor', 'Quality Measure']
    return Dist_stat


In [68]:
year_SRC = SRC_perform(df_year_adj)
year_Pbias = Pbias_perform(df_year_adj)
year_MEF = MEF_perform(df_year_adj)
year_CF = CF_perform(df_year_adj)
year_R2 = R2_perform(df_year_adj)

sumwint_Pbias = Pbias_perform(df_sumwint)        
sumwint_SRC = SRC_perform(df_sumwint)
sumwint_MEF = MEF_perform(df_sumwint)
sumwint_CF = CF_perform(df_sumwint)
sumwint_R2 = R2_perform(df_sumwint)

### Model Performance

The model performance for all measurements evaluated against five quality measures is presented in the table below

In [69]:
all_Pbias = Pbias_perform(df_all)
all_SRC = SRC_perform(df_all)
all_MEF = MEF_perform(df_all)
all_CF = CF_perform(df_all)
all_R2 = R2_perform(df_all)

# all data all measures:
all_stats = all_Pbias.append(all_SRC, ignore_index = True)
all_stats = all_stats.append(all_MEF, ignore_index = True)
all_stats = all_stats.append(all_CF, ignore_index = True)
all_stats = all_stats.append(all_R2, ignore_index = True)

# all stats performance measure:
DistAll = distribution(all_stats,'All Quality Measures')
DistAll.style.hide_index()

Excellent,Very good,Good,Poor,Above poor,Quality Measure
8.2,30.9,24.5,36.4,63.64,All Quality Measures


### Model performance above poor for each quality measure

Model performance for all measurements evaluated from each of the quality measure:

In [70]:
DistQmeas = distribution(all_Pbias,'P_Bias')
tmp = distribution(all_SRC,'SRC')
DistQmeas = DistQmeas.append(tmp, ignore_index = True)
tmp = distribution(all_MEF,'MEF')
DistQmeas = DistQmeas.append(tmp, ignore_index = True)
tmp = distribution(all_CF,'CF')
DistQmeas = DistQmeas.append(tmp, ignore_index = True)
tmp = distribution(all_R2,'R2')
DistQmeas = DistQmeas.append(tmp, ignore_index = True)
DistQmeas.style.hide_index()

Excellent,Very good,Good,Poor,Above poor,Quality Measure
13.6,4.5,45.5,36.4,63.64,P_Bias
0,45.5,18.2,36.4,63.64,SRC
0,13.6,13.6,72.7,27.27,MEF
0,68.2,31.8,0,100,CF
27.3,22.7,13.6,36.4,63.64,R2


### Model performance for each station

Model performance for all quality measures for each station, percentage above poor:

In [71]:
stations = df_all['Station'].unique()
DistStations = pd.DataFrame()
for station in stations:
    station_stats = all_stats[all_stats['Station']==station]
    tmp1 = distribution(station_stats,'All Q measures')
    tmp1['Station'] = station
    DistStations = DistStations.append(tmp1, ignore_index = True)
DistStations.style.hide_index()

Excellent,Very good,Good,Poor,Above poor,Quality Measure,Station
9.1,30.9,21.8,38.2,61.82,All Q measures,RKB1
7.3,30.9,27.3,34.5,65.45,All Q measures,RKB10


### Model performance, all parameters

In [72]:
stations = df_all['Station'].unique()
from ipywidgets import interact
if len(df_all['Station'].unique()) <=5:
    print('Here you will find a summary of modelperformance. Use the drop down function to select the different stations used for calibration/validation')
    @interact
    def show_ts(Station = stations):
        station_table = df_all.loc[df_all['Station']==Station]
        station_table = station_table.drop(['Station','Standard','Summer','Winter'], axis=1)
        station_table['Percent_Bias'] = round(station_table['Percent_Bias'],1)
        station_table['Spearman_Rank_Correlation'] = round(station_table['Spearman_Rank_Correlation'],2)
        station_table['Efficiency_Factor'] = round(station_table['Efficiency_Factor'],2)
        station_table['Cost_Function'] = round(station_table['Cost_Function'],2)
        station_table['R2'] = round(station_table['R2'],2)

        return station_table.style.hide_index()

Here you will find a summary of modelperformance. Use the drop down function to select the different stations used for calibration/validation


interactive(children=(Dropdown(description='Station', options=('RKB1', 'RKB10'), value='RKB1'), Output()), _do…

In [73]:
if len(df_all['Station'].unique()) >5:
    Qmeas = ['Percent_Bias','Spearman_Rank_Correlation','Efficiency_Factor','Cost_Function','R2']
    print('Here you will find a summary of modelperformance. Use the drop down function to select the different quality measure')
    @interact
    def show_ts(QMeasure=Qmeas):
        station_Meas = df_all['Measurement'].unique()
        Stat_table = pd.DataFrame()
        for station in df_all['Station'].unique():
            tmp = df_all.drop(['Standard','Summer','Winter'], axis=1)
            station_tab = tmp.loc[tmp['Station']==station]
            station_Obs = station_tab['Total_Number_Of_Observations_1']
            station_stat = pd.DataFrame(station_tab[QMeasure]).T
            station_stat.columns = station_Meas
            min_obs = min(station_Obs)
            max_obs= max(station_Obs)
            station_stat['min. no. obs.'] = min_obs
            station_stat['max. no. obs.'] = max_obs
            station_stat['station'] = station
            Stat_table = Stat_table.append(station_stat, ignore_index = True)
        return Stat_table.style.hide_index()

In [74]:
#Qmeas = ['Percent_Bias','Spearman_Rank_Correlation','Efficiency_Factor','Cost_Function','R2']
#station_Meas = df_all['Measurement'].unique()

#@interact
#def show_ts(QMeasure=Qmeas):
#    Stat_table = pd.DataFrame()
#    for station in df_all['Station'].unique():
#        tmp = df_all.drop(['Standard','Summer','Winter'], axis=1)
#        station_tab = tmp.loc[tmp['Station']==station]
#        station_Obs = station_tab['Total_Number_Of_Observations_1']
#        station_stat = pd.DataFrame(round(station_tab[QMeasure],2)).T
#        station_stat.columns = station_Meas
#        min_obs = min(station_Obs)
#        max_obs= max(station_Obs)
#        station_stat['min. no. obs.'] = min_obs
#        station_stat['max. no. obs.'] = max_obs
#        station_stat['station'] = station
#        Stat_table = Stat_table.append(station_stat, ignore_index = True)
#    return Stat_table.style.hide_index()

### Average model performance for each parameter

In [75]:
def performance_param_dist(data):
    qmeasure = data['QMeasure'].unique()
    ave_perform = pd.DataFrame()
    for i in range(len(qmeasure)):
        data_QM = data[data['QMeasure']==qmeasure[i]]
        #param_stat = collect_param_df(data_QM, param)
        ave = np.mean(data_QM['Stat_Value'])
        #ave = np.mean(param_stat['Stat_Value'])
        if qmeasure[i] == 'P_Bias':
            if ave <10:
                tmp = pd.DataFrame([round(ave,1), 'Excellent', 'Ave_P_bias']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True)
            if ave >=10 and ave <=20:
                tmp = pd.DataFrame([round(ave,1), 'Very good', 'Ave_P_bias']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True)
            if ave >20 and ave <=40:
                tmp = pd.DataFrame([round(ave,1), 'Good', 'Ave_P_bias']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True)
            if ave >40:
                tmp = pd.DataFrame([round(ave,1), 'Poor', 'Ave_P_bias']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True)
              
        if qmeasure[i] == 'SRC':
            if ave >0.9:
                tmp = pd.DataFrame([round(ave,1), 'Excellent', 'Ave_SRC']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True)
            if ave <=0.9 and ave >=0.6:
                tmp = pd.DataFrame([round(ave,1), 'Very good', 'Ave_SRC']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True)
            if ave <0.6 and ave >=0.3:
                tmp = pd.DataFrame([round(ave,1), 'Good', 'Ave_SRC']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True) 
            if ave <0.3:
                tmp = pd.DataFrame([round(ave,1), 'Poor', 'Ave_SRC']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True)
                
        if qmeasure[i] == 'MEF':
            if ave >0.8:
                tmp = pd.DataFrame([round(ave,1), 'Excellent', 'Ave_MEF']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True)
            if ave <=0.8 and ave >=0.5:
                tmp = pd.DataFrame([round(ave,1), 'Very good', 'Ave_MEF']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True)
            if ave <0.5 and ave >=0.2:
                tmp = pd.DataFrame([round(ave,1), 'Good', 'Ave_MEF']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True)
            if ave <0.2:
                tmp = pd.DataFrame([round(ave,1), 'Poor', 'Ave_MEF']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True)
                
        if qmeasure[i] == 'CF':
            if ave <=0.4:
                tmp = pd.DataFrame([round(ave,1), 'Excellent', 'Ave_CF']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True)
            if ave >0.4 and ave <=1:
                tmp = pd.DataFrame([round(ave,1), 'Very good', 'Ave_CF']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True)
            if ave >1 and ave <3:
                tmp = pd.DataFrame([round(ave,1), 'Good', 'Ave_CF']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True) 
            if ave >=3:
                tmp = pd.DataFrame([round(ave,1), 'Poor', 'Ave_CF']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True)
        #R2
        if qmeasure[i] == 'R2':
            if ave >0.65:
                tmp = pd.DataFrame([round(ave,1), 'Excellent', 'Ave_R2']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True)
            if ave <=0.65 and ave >=0.35:
                tmp = pd.DataFrame([round(ave,1), 'Very good', 'Ave_R2']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True)
            if ave <0.35 and ave >=0.175:
                tmp = pd.DataFrame([round(ave,1), 'Good', 'Ave_R2']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True)  
            if ave <0.175:
                tmp = pd.DataFrame([round(ave,1), 'Poor', 'Ave_R2']).T
                ave_perform = ave_perform.append(tmp, ignore_index = True)
    ave_perform.columns = ['Ave. Stat. Value','Performance','Q measure']
        # average parameter performance
    return ave_perform

In [76]:
CH_Performance = collect_param_df(all_Pbias, 'CH')
tmp1 = collect_param_df(all_SRC, 'CH')
CH_Performance = CH_Performance.append(tmp1)
tmp2 = collect_param_df(all_MEF, 'CH')
CH_Performance = CH_Performance.append(tmp2)
tmp3 = collect_param_df(all_CF, 'CH')
CH_Performance = CH_Performance.append(tmp3)
tmp4 = collect_param_df(all_R2, 'CH')
CH_Performance = CH_Performance.append(tmp4)

CH_ave_QMeas = performance_param_dist(CH_Performance)
CH_ave_QMeas['Param']='CH'

CH_Performance = collect_param_df(year_Pbias, 'CH')
tmp1 = collect_param_df(year_SRC, 'CH')
CH_Performance = CH_Performance.append(tmp1)
tmp2 = collect_param_df(year_MEF, 'CH')
CH_Performance = CH_Performance.append(tmp2)
tmp3 = collect_param_df(year_CF, 'CH')
CH_Performance = CH_Performance.append(tmp3)
tmp4 = collect_param_df(year_R2, 'CH')
CH_Performance = CH_Performance.append(tmp4)

CH_ave_ann_QMeas = performance_param_dist(CH_Performance)
CH_ave_ann_QMeas['Param']='CH'
CH_ave_ann_QMeas['Season']='Annual'

DO_Perform = collect_param_df(all_Pbias, 'DO')
tmp = collect_param_df(all_SRC, 'DO')
DO_Perform = DO_Perform.append(tmp)
tmp = collect_param_df(all_MEF, 'DO')
DO_Perform = DO_Perform.append(tmp)
tmp = collect_param_df(all_CF, 'DO')
DO_Perform = DO_Perform.append(tmp)
tmp = collect_param_df(all_R2, 'DO')
DO_Perform = DO_Perform.append(tmp)

DO_ave_QMeas = performance_param_dist(DO_Perform)
DO_ave_QMeas['Param']='DO'

DIN_Perform = collect_param_df(all_Pbias, 'DIN')
tmp = collect_param_df(all_SRC, 'DIN')
DIN_Perform = DIN_Perform.append(tmp)
tmp = collect_param_df(all_MEF, 'DIN')
DIN_Perform = DIN_Perform.append(tmp)
tmp = collect_param_df(all_CF, 'DIN')
DIN_Perform = DIN_Perform.append(tmp)
tmp = collect_param_df(all_R2, 'DIN')
DIN_Perform = DIN_Perform.append(tmp)

DIN_ave_QMeas = performance_param_dist(DIN_Perform)
DIN_ave_QMeas['Param']='DIN'

DIN_Perform = collect_param_df(year_Pbias, 'DIN')
tmp = collect_param_df(year_SRC, 'DIN')
DIN_Perform = DIN_Perform.append(tmp)
tmp = collect_param_df(year_MEF, 'DIN')
DIN_Perform = DIN_Perform.append(tmp)
tmp = collect_param_df(year_CF, 'DIN')
DIN_Perform = DIN_Perform.append(tmp)
tmp = collect_param_df(year_R2, 'DIN')
DIN_Perform = DIN_Perform.append(tmp)

DIN_ave_ann_QMeas = performance_param_dist(DIN_Perform)
DIN_ave_ann_QMeas['Param']='DIN'
DIN_ave_ann_QMeas['Season']='Annual'

PO4_Perform = collect_param_df(all_Pbias, 'PO4')
tmp = collect_param_df(all_SRC, 'PO4')
PO4_Perform = PO4_Perform.append(tmp)
tmp = collect_param_df(all_MEF, 'PO4')
PO4_Perform = PO4_Perform.append(tmp)
tmp = collect_param_df(all_CF, 'PO4')
PO4_Perform = PO4_Perform.append(tmp)
tmp = collect_param_df(all_R2, 'PO4')
PO4_Perform = PO4_Perform.append(tmp)

PO4_ave_QMeas = performance_param_dist(PO4_Perform)
PO4_ave_QMeas['Param']='PO4'

PO4_Perform = collect_param_df(year_Pbias, 'PO4')
tmp = collect_param_df(year_SRC, 'PO4')
PO4_Perform = PO4_Perform.append(tmp)
tmp = collect_param_df(year_MEF, 'PO4')
PO4_Perform = PO4_Perform.append(tmp)
tmp = collect_param_df(year_CF, 'PO4')
PO4_Perform = PO4_Perform.append(tmp)
tmp = collect_param_df(year_R2, 'PO4')
PO4_Perform = PO4_Perform.append(tmp)

PO4_ave_ann_QMeas = performance_param_dist(PO4_Perform)
PO4_ave_ann_QMeas['Param']='PO4'
PO4_ave_ann_QMeas['Season']='Annual'

KD_Perform = collect_param_df(all_Pbias, 'KD')
tmp = collect_param_df(all_SRC, 'KD')
KD_Perform = DO_Perform.append(tmp)
tmp = collect_param_df(all_MEF, 'KD')
KD_Perform = DO_Perform.append(tmp)
tmp = collect_param_df(all_CF, 'KD')
KD_Perform = DO_Perform.append(tmp)
tmp = collect_param_df(all_R2, 'KD')
KD_Perform = KD_Perform.append(tmp)

KD_ave_QMeas = performance_param_dist(KD_Perform)
KD_ave_QMeas['Param']='KD'

KD_Perform = collect_param_df(year_Pbias, 'KD')
tmp = collect_param_df(year_SRC, 'KD')
KD_Perform = DO_Perform.append(tmp)
tmp = collect_param_df(year_MEF, 'KD')
KD_Perform = DO_Perform.append(tmp)
tmp = collect_param_df(year_CF, 'KD')
KD_Perform = DO_Perform.append(tmp)
tmp = collect_param_df(year_R2, 'KD')
KD_Perform = KD_Perform.append(tmp)

KD_ave_ann_QMeas = performance_param_dist(KD_Perform)
KD_ave_ann_QMeas['Param']='KD'
KD_ave_ann_QMeas['Season']='Annual'

TN_Perform = collect_param_df(all_Pbias, 'TN')
tmp = collect_param_df(all_SRC, 'TN')
TN_Perform = TN_Perform.append(tmp)
tmp = collect_param_df(all_MEF, 'TN')
TN_Perform = TN_Perform.append(tmp)
tmp = collect_param_df(all_CF, 'TN')
TN_Perform = TN_Perform.append(tmp)
tmp = collect_param_df(all_R2, 'TN')
TN_Perform = TN_Perform.append(tmp)

TN_ave_QMeas = performance_param_dist(TN_Perform)
TN_ave_QMeas['Param']='TN'

TP_Perform = collect_param_df(all_Pbias, 'TP')
tmp = collect_param_df(all_SRC, 'TP')
TP_Perform = TP_Perform.append(tmp)
tmp = collect_param_df(all_MEF, 'TP')
TP_Perform = TP_Perform.append(tmp)
tmp = collect_param_df(all_CF, 'TP')
TP_Perform = TP_Perform.append(tmp)
tmp = collect_param_df(all_R2, 'TP')
TP_Perform = TP_Perform.append(tmp)

TP_ave_QMeas = performance_param_dist(TP_Perform)
TP_ave_QMeas['Param']='TP'

In [ ]:
meas = ['Annual','All']

par = ['CH','DO','KD','DIN','DIP','TN','TP']
test = pd.DataFrame([0])
test['Annual']=1
test['All']=0
                        
@interact
def show_ts(Parameter=par,Measurements=meas):
    check = test[Measurements]
    if check[0] == 0:
        perform = collect_param_df(all_Pbias, Parameter)
        tmp = collect_param_df(all_SRC, Parameter)
        perform = perform.append(tmp)
        tmp = collect_param_df(all_MEF, Parameter)
        perform = perform.append(tmp)
        tmp = collect_param_df(all_CF, Parameter)
        perform = perform.append(tmp)
        tmp = collect_param_df(all_R2, Parameter)
        perform = perform.append(tmp)

        ave_QMeas = performance_param_dist(perform)
        ave_QMeas['Param']=Parameter

    if check[0] == 1:
        perform = collect_param_df(year_Pbias, Parameter)
        tmp = collect_param_df(year_SRC, Parameter)
        perform = perform.append(tmp)
        tmp = collect_param_df(year_MEF, Parameter)
        perform = perform.append(tmp)
        tmp = collect_param_df(year_CF, Parameter)
        perform = perform.append(tmp)
        tmp = collect_param_df(year_R2, Parameter)
        perform = perform.append(tmp)

        ave_QMeas = performance_param_dist(perform)
        ave_QMeas['Param']=Parameter

    return ave_QMeas